# Eight Queens Genetic Algorithim

In [1]:
import random

from scipy import special

def generate_board_state(number_of_queens):
    return [random.randrange(number_of_queens) for num in range(number_of_queens )]

def generate_population(number_of_queens=8, size_of_population=19):
        return [generate_board_state(number_of_queens) for _ in range(size_of_population)]

def check_for_completed_board(boards, number_of_queens, perfect_score):
    for board in boards:
        if generate_fitness_score(board, number_of_queens) == perfect_score:
            return board

    return []


In [2]:
def generate_fitness_score(board_state, number_of_queens):
    """determine number of queens not being attacked
        :input board_state array of queens
    """
    fitness_score = 0

    for determining_row_index in range(number_of_queens):
        determining_value = board_state[determining_row_index]
        for row_index in range(number_of_queens):
            # can't attack itself
            if determining_row_index == row_index:
                continue
            if board_state[row_index] == determining_value:
                continue
            if row_index + board_state[row_index] == determining_row_index + determining_value:
                continue
            if row_index - board_state[row_index] == determining_row_index - determining_value:
                continue

            fitness_score += 1

    return fitness_score / 2


## test genereate fitness score
generate_fitness_score([7,1,4,2,0,6,3,5], number_of_queens=8) == 28

True

In [3]:
def _random_weighted_choice(population, weights, selected=None):
    """pick one from list proportional to weights"""

    if selected is not None:
        index = population.index(selected)
        population = population[:index] + population[index+1:]
        weights = weights[:index] + weights[index+1:]

    return random.choices(population, weights=weights, k=1)[0]


def selection(boards, fitness_scores, size_of_population):
    """return a list of selected pairs"""
    # create weights

    selected = []
    for _ in range(size_of_population):
        parent_one = _random_weighted_choice(boards, fitness_scores)
        parent_two = _random_weighted_choice(boards, fitness_scores, selected=parent_one)

        selected.append((parent_one, parent_two))

    return selected



test_population = [[1],[2],]
weights = [1,1]

selection(test_population, weights, size_of_population=2)

test_population = [[1],[2],[3],[4]]
weights = [19,20,26,16]

selection(test_population, weights, size_of_population=4)


[([1], [4]), ([3], [1]), ([2], [4]), ([4], [2])]

In [4]:

def _crssover(parent_one, parent_two, cross_over_index):
    return parent_one[:cross_over_index] + parent_two[cross_over_index:]

def crossover(selected_pairs, number_of_queens):

    children = []
    for s_p in selected_pairs:
        # generate random cross over points - 0 - max number of queens
        cross_over_index = random.randint(0, number_of_queens - 1)
        parent_one = s_p[0]
        parent_two = s_p[1]   

        child = _crssover(parent_one, parent_two, cross_over_index)
        children.append(child)

    return children


crossover([([1,1,1,1,1,1,1,1], [0,0,0,0,0,0,0,0])], number_of_queens=8)

[[1, 1, 1, 1, 1, 1, 0, 0]]

In [59]:
def _mutate(board, number_of_queens):
    mutation_point = random.randint(0, number_of_queens - 1)
    # TODO this could possibly be the same value

    current_value = board[mutation_point]
    new_value = current_value

    while new_value == current_value:
        new_value = random.randint(0, number_of_queens - 1)

    board[mutation_point] = new_value

    return board


def mutation(boards, mutation_frequency, number_of_queens):        
    for i,board in enumerate(boards):
        x = random.randint(1, mutation_frequency)
        if x == 1:
            boards[i] = _mutate(board, number_of_queens)
    return boards


_mutate([1,2,3], 3)
mutation([[1,2,3]], 3, 3)

[[1, 2, 3]]

In [60]:
def genetic_algorithm(population, size_of_population, number_of_queens, mutation_frequency):
    # b) fitness function
    # a fitness score is calculated based off the number of non attacking queen pairs
    fit_scores = [generate_fitness_score(board, number_of_queens) for board in population]

    # fitness scores

    # c) select parents
    # two parents are selected at random - in accordance with probability from score in b)
    # a cross over point is chosen at random from positions in the string
    selected_pairs = selection(population, fit_scores, size_of_population)
    
    # d) cross over
    # offspring are created by crossing over parent strings at the crossover point
    # crossover(selected_pairs,number_of_queens)
    children = crossover(selected_pairs, number_of_queens)

    mutated = mutation(children, mutation_frequency, number_of_queens)

    return mutated, fit_scores

    # e) mutation
        # each location is subject to random mutation with a small independent probability
        # this involves choosing a queen at random and moving it to a random square in its column

    # f) TODO is this a thing?????
    # introduce previous top scoring offspring to keep fitness ??? is this part of original algo?? or an optimisation????


In [62]:
test_population = [[7, 6, 3, 2, 6, 1, 1, 5],
 [3, 0, 3, 6, 2, 6, 6, 0],
 [5, 4, 1, 6, 1, 3, 7, 1],
 [7, 7, 2, 2, 1, 4, 2, 7],
 [0, 3, 3, 4, 6, 0, 5, 7],
 [7, 2, 5, 5, 1, 7, 5, 0],
 [6, 5, 4, 1, 0, 5, 6, 4],
 [6, 7, 3, 7, 4, 4, 6, 4],
 [7, 2, 7, 0, 4, 2, 1, 3],
 [1, 6, 7, 4, 1, 7, 3, 7],
 [7, 4, 0, 6, 2, 4, 4, 7],
 [0, 5, 6, 3, 6, 4, 0, 0],
 [4, 1, 0, 3, 0, 4, 3, 1],
 [1, 0, 6, 3, 5, 1, 5, 4],
 [2, 6, 7, 3, 2, 4, 7, 4],
 [7, 5, 7, 2, 2, 0, 0, 0],
 [0, 3, 3, 6, 0, 3, 1, 2],
 [0, 7, 7, 4, 3, 3, 2, 5],
 [3, 1, 7, 6, 3, 5, 6, 0]]

NUMBER_OF_QUEENS=8
SIZE_OF_POPULATION=19
MUTATION_FREQUENCY = 100

# a) generate initial population
population = test_population
## TODO - how is this calculated??
perfect_score = special.comb(NUMBER_OF_QUEENS, 2)
completed_board = check_for_completed_board(population, NUMBER_OF_QUEENS, perfect_score)
new_population, fitness_scores = genetic_algorithm(population, SIZE_OF_POPULATION, NUMBER_OF_QUEENS, MUTATION_FREQUENCY)
population = new_population

population

[[6, 7, 2, 2, 1, 4, 2, 7],
 [1, 0, 6, 3, 6, 4, 0, 0],
 [3, 0, 3, 6, 2, 6, 5, 0],
 [6, 6, 3, 2, 6, 1, 1, 5],
 [7, 2, 5, 4, 1, 7, 3, 7],
 [3, 1, 7, 6, 3, 5, 7, 4],
 [7, 4, 0, 6, 2, 4, 4, 7],
 [3, 1, 7, 3, 2, 4, 7, 4],
 [3, 0, 3, 6, 2, 6, 6, 0],
 [0, 7, 7, 4, 2, 4, 7, 4],
 [6, 5, 4, 1, 1, 4, 2, 7],
 [6, 7, 3, 7, 4, 4, 6, 0],
 [1, 6, 7, 4, 3, 3, 2, 5],
 [4, 1, 7, 4, 1, 7, 3, 7],
 [4, 1, 0, 3, 0, 4, 2, 5],
 [0, 3, 3, 4, 1, 7, 3, 7],
 [1, 6, 7, 4, 1, 7, 3, 7],
 [7, 4, 0, 6, 2, 6, 6, 0],
 [0, 3, 3, 6, 0, 3, 1, 0]]

In [78]:
NUMBER_OF_QUEENS=8
SIZE_OF_POPULATION=19
MUTATION_FREQUENCY = 20

# a) generate initial population
population = generate_population(NUMBER_OF_QUEENS, SIZE_OF_POPULATION)
perfect_score = special.comb(NUMBER_OF_QUEENS, 2)

In [79]:
gen = 0
completed_board = []

# # g) check if it has found correct state and exit loop
while not completed_board:
    population, fitness_scores = genetic_algorithm(population, SIZE_OF_POPULATION, NUMBER_OF_QUEENS, MUTATION_FREQUENCY)
    completed_board = check_for_completed_board(population, NUMBER_OF_QUEENS, perfect_score)
    gen += 1

    if gen == 10000:
        break
    if gen==1 or gen % 1000 == 0:
        print(f"run for {gen} generations")
        print(population)
        print(fitness_scores)

print(f"solved in {gen} generations")
print(f"completed board: {completed_board}")

run for 1 generations
[[3, 7, 2, 0, 2, 6, 3, 7], [5, 0, 0, 7, 2, 5, 1, 6], [7, 0, 4, 2, 2, 5, 1, 6], [5, 0, 0, 7, 2, 1, 7, 6], [3, 6, 2, 0, 0, 1, 7, 7], [7, 6, 0, 0, 3, 2, 1, 2], [0, 5, 0, 5, 0, 3, 2, 1], [3, 7, 3, 0, 2, 5, 6, 0], [4, 0, 3, 2, 0, 1, 2, 0], [7, 0, 4, 2, 2, 3, 5, 4], [2, 3, 6, 7, 1, 7, 3, 7], [3, 5, 1, 0, 3, 2, 1, 2], [5, 0, 1, 0, 5, 5, 2, 0], [2, 3, 0, 5, 0, 3, 2, 0], [3, 3, 0, 0, 4, 0, 6, 7], [5, 6, 0, 6, 4, 2, 6, 0], [7, 6, 0, 1, 3, 3, 6, 0], [3, 3, 0, 0, 4, 0, 2, 0], [2, 3, 6, 7, 1, 7, 1, 2]]
[20.0, 23.0, 19.0, 17.0, 21.0, 19.0, 22.0, 20.0, 23.0, 20.0, 18.0, 21.0, 22.0, 14.0, 22.0, 18.0, 16.0, 21.0, 20.0]
run for 1000 generations
[[1, 7, 0, 3, 3, 5, 0, 5], [5, 7, 0, 3, 3, 5, 1, 6], [5, 7, 0, 3, 3, 5, 0, 4], [1, 7, 0, 3, 3, 5, 1, 4], [1, 7, 0, 3, 3, 5, 0, 4], [1, 7, 0, 3, 3, 5, 0, 4], [5, 7, 0, 3, 3, 5, 0, 4], [1, 7, 0, 3, 3, 5, 0, 4], [7, 7, 0, 3, 3, 5, 0, 4], [5, 7, 0, 3, 3, 5, 1, 4], [5, 7, 0, 3, 3, 5, 1, 6], [5, 7, 7, 3, 3, 5, 0, 4], [1, 7, 0, 3, 3, 5, 0, 4], [1, 